In [13]:
from utils import data_dir
import pandas as pd
from scripts.count_words import  XML_DOCUMENT_SCHEMA_MAP
from lxml import etree

In [14]:
agencies_filename = data_dir / 'agencies.json'
df = pd.read_json(agencies_filename)
child_dfs = []
for i, row in df.iterrows():
    agency = row["name"]
    children_df = pd.DataFrame(row["children"])
    if len(children_df) > 0:
        children_df["parent_agency"] = agency
        child_dfs.append(children_df)

child_dfs = pd.concat(child_dfs, ignore_index=True)
df["parent_agency"] = df["name"]
all_df = pd.concat([df, child_dfs], ignore_index=True)

In [15]:
all_df['cfr_references']

0                       [{'title': 1, 'chapter': 'III'}]
1                     [{'title': 36, 'chapter': 'VIII'}]
2                   [{'title': 5, 'chapter': 'LXXXIII'}]
3      [{'title': 22, 'chapter': 'XV'}, {'title': 48,...
4      [{'title': 22, 'chapter': 'V'}, {'title': 48, ...
                             ...                        
311                      [{'title': 31, 'chapter': 'I'}]
312                    [{'title': 12, 'chapter': 'XVI'}]
313                      [{'title': 31, 'chapter': 'V'}]
314                   [{'title': 31, 'chapter': 'VIII'}]
315                     [{'title': 31, 'subtitle': 'A'}]
Name: cfr_references, Length: 316, dtype: object

In [34]:
from scripts.download_versions import RAW_VERSIONS_PATH
import json

for title_number in range(1, 2):
    print(f"title {title_number}")
    with open(data_dir / f"raw_title_xml/title_{title_number}.xml", "r") as f:
        xml_string = f.read()
    parser = etree.XMLParser(recover=True)  # More forgiving parser  # type: ignore 
    root = etree.fromstring(xml_string.encode("utf-8"), parser) 

    file_path = RAW_VERSIONS_PATH / f"title_{title_number}_changes.json"
    with open(file_path, "r") as file:
        data = json.load(file)

    for json_path in data["content_versions"]:
        print(json_path)
        part = json_path["part"]
        type = XML_DOCUMENT_SCHEMA_MAP['part']["type"]
        # Find the part (DIV5)
        xpath = f".//DIV5[@N='{part}']"
        elements = root.xpath(xpath)
        if len(elements) == 0:
            print(f"Element not found: {xpath}")
            continue
        if len(elements) > 1:
            print(f"Multiple elements found: {xpath}")
        section_element = elements[0]
        current_element = section_element
        row = {}
        while current_element.get("TYPE") != "TITLE":
            row[current_element.get("TYPE")] = current_element.get("N")
            current_element = current_element.getparent()

title 1
{'date': '2015-12-18', 'amendment_date': '2015-12-18', 'issue_date': '2015-12-18', 'identifier': '1.1', 'name': '§ 1.1   Definitions.', 'part': '1', 'substantive': True, 'removed': False, 'subpart': None, 'title': '1', 'type': 'section'}
{'date': '2015-12-18', 'amendment_date': '2015-12-18', 'issue_date': '2015-12-18', 'identifier': '2.2', 'name': '§ 2.2   Administrative Committee of the Federal Register.', 'part': '2', 'substantive': True, 'removed': False, 'subpart': None, 'title': '1', 'type': 'section'}
{'date': '2015-12-18', 'amendment_date': '2015-12-18', 'issue_date': '2015-12-18', 'identifier': '10.2', 'name': '§ 10.2   Scope and sources.', 'part': '10', 'substantive': True, 'removed': False, 'subpart': 'A', 'title': '1', 'type': 'section'}
{'date': '2015-12-18', 'amendment_date': '2015-12-18', 'issue_date': '2015-12-18', 'identifier': '9.2', 'name': '§ 9.2   Scope.', 'part': '9', 'substantive': True, 'removed': False, 'subpart': None, 'title': '1', 'type': 'section'}
{

In [35]:
import streamlit as st
from st_files_connection import FilesConnection

# Create connection object and retrieve file contents.
# Specify input format is a csv and to cache the result for 600 seconds.
conn = st.connection('s3', type=FilesConnection)
df = conn.read("testbucket-jrieke/myfile.csv", input_format="csv", ttl=600)


2025-04-17 08:13:42.213 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 08:13:42.256 
  command:

    streamlit run /Users/luke/self/eCFR-analyzer/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-17 08:13:42.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 08:13:42.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 08:13:42.883 Thread 'Thread-169': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 08:13:42.886 Thread 'Thread-169': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 08:13:44.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 08:13:44.029 

NoCredentialsError: Unable to locate credentials

In [1]:
from utils import  get_or_load_titles_list

In [2]:
t =  get_or_load_titles_list()

In [4]:
{tt['number']: tt['name'] for tt in t['titles']}

{1: 'General Provisions',
 2: 'Federal Financial Assistance',
 3: 'The President',
 4: 'Accounts',
 5: 'Administrative Personnel',
 6: 'Domestic Security',
 7: 'Agriculture',
 8: 'Aliens and Nationality',
 9: 'Animals and Animal Products',
 10: 'Energy',
 11: 'Federal Elections',
 12: 'Banks and Banking',
 13: 'Business Credit and Assistance',
 14: 'Aeronautics and Space',
 15: 'Commerce and Foreign Trade',
 16: 'Commercial Practices',
 17: 'Commodity and Securities Exchanges',
 18: 'Conservation of Power and Water Resources',
 19: 'Customs Duties',
 20: "Employees' Benefits",
 21: 'Food and Drugs',
 22: 'Foreign Relations',
 23: 'Highways',
 24: 'Housing and Urban Development',
 25: 'Indians',
 26: 'Internal Revenue',
 27: 'Alcohol, Tobacco Products and Firearms',
 28: 'Judicial Administration',
 29: 'Labor',
 30: 'Mineral Resources',
 31: 'Money and Finance: Treasury',
 32: 'National Defense',
 33: 'Navigation and Navigable Waters',
 34: 'Education',
 35: 'Reserved',
 36: 'Parks, For